In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Paralelizable

In [ ]:
def Metrica(Par):
  x=rouge.get_scores(Par[1], Par[0])[0]
  metrica=[]
  metrica.append((x['rouge-1']['r']+x['rouge-1']['f']+x['rouge-1']['p'])/3)
  metrica.append((x['rouge-2']['r']+x['rouge-2']['f']+x['rouge-2']['p'])/3)
  metrica.append((x['rouge-l']['r']+x['rouge-l']['f']+x['rouge-l']['p'])/3)
  return(metrica)

Con un ciclo usual

In [ ]:
def MetricaFor(Pares):
  metrica=[]
  i=0
  for Par in Pares:
    if (type(Par[0])==str) & (type(Par[1])==str):
      try:
        x=rouge.get_scores(Par[1], Par[0])[0]
        M1=(x['rouge-1']['r']+x['rouge-1']['f']+x['rouge-1']['p'])/3
        M2=((x['rouge-2']['r']+x['rouge-2']['f']+x['rouge-2']['p'])/3)
        M3=((x['rouge-l']['r']+x['rouge-l']['f']+x['rouge-l']['p'])/3)
      except: ()
    metrica.append([M1,M2,M3])
  return(metrica)

In [ ]:
import pandas as pd
import numpy as np

## Baseline

In [ ]:
Baseline_resultados=pd.read_csv('/content/drive/Othercomputers/DESKTOP/D:/Computo Estadistico/semestre 3/ciencia de datos 2/piav2/newresumenbaseline.csv',sep=';',on_bad_lines='skip')
Baseline_resultados.head()

In [ ]:
Baseline_resultados=Baseline_resultados.drop(columns='Unnamed: 0')
Baseline_resultados.head()

In [ ]:
Baseline=Baseline_resultados.sample(10000, random_state=2022)
Baseline.head()

In [ ]:
Baseline=Baseline.reset_index().drop(columns='index')

In [ ]:
pares=[]
for i in range(Baseline.shape[0]):
  pares.append([Baseline['newresumen'][i], Baseline['resumen'][i]])

In [ ]:
Metrica_Baseline=MetricaFor(pares)

In [ ]:
r1,r2,r3=0,0,0
for i in range(Baseline.shape[0]):
  try:
    r1+=Metrica_Baseline[i][0]
    r2+=Metrica_Baseline[i][1]
    r3+=Metrica_Baseline[i][2]
  except: ()

metrica_prueba=np.array([r1/(Baseline.shape[0]), r2/(Baseline.shape[0]), r3/(Baseline.shape[0])])

In [ ]:
metrica_prueba

array([0.24602552, 0.0227704 , 0.0227704 ])

##LSTM

In [ ]:
LSTM_resultados=pd.read_csv('/content/drive/MyDrive/Resultados Resumen/LSTM-Corregido/BaseLSTMCorregido.csv')

In [ ]:
LSTM_resultados=LSTM_resultados.drop(columns='Unnamed: 0')
LSTM_resultados.head()

In [ ]:
LSTM=LSTM_resultados.sample(10000, random_state=2022)
LSTM.head()

In [ ]:
LSTM=LSTM.reset_index().drop(columns='index')

In [ ]:
#resumen=[generate_summary(texto) for texto in df['Procesado']]
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
# install findspark using pip
!pip install -q findspark

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

import findspark
findspark.init()


In [ ]:
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
pares=[]
for i in range(LSTM.shape[0]):
  pares.append([LSTM['ResumenNew'][i], LSTM['Resumenes'][i]])

In [ ]:
LSTM_rdd=sc.parallelize(pares)

In [ ]:
Metrica_LSTM=LSTM_rdd.map(Metrica).collect()

In [ ]:
r1,r2,r3=0,0,0
for i in range(LSTM.shape[0]):
  r1+=Metrica_LSTM[i][0]
  r2+=Metrica_LSTM[i][1]
  r3+=Metrica_LSTM[i][2]

metrica_prueba=np.array([r1/LSTM.shape[0]), r2/(LSTM.shape[0]), r3/(LSTM.shape[0])])

In [ ]:
metrica_prueba

array([0.14244326, 0.01306064, 0.09241681])

##GRU

In [ ]:
GRU_resultados=pd.read_csv('/content/drive/MyDrive/Resultados Resumen/GRU-Faltan Iteraciones/BaseGRUFalta.csv')

In [ ]:
GRU_resultados=GRU_resultados.drop(columns='Unnamed: 0')
GRU_resultados.head()

In [ ]:
GRU=GRU_resultados.sample(10000, random_state=2022)
GRU.head()

In [ ]:
GRU=GRU[GRU['Resumenes'].notna()]
GRU=GRU.reset_index().drop(columns='index')

In [ ]:
pares=[]
for i in range(GRU.shape[0]):
  pares.append([GRU['ResumenNew'][i], GRU['Resumenes'][i]])

In [ ]:
GRU_rdd=sc.parallelize(pares)


In [ ]:
Metrica_GRU=GRU_rdd.map(Metrica).collect()

In [ ]:
r1,r2,r3=0,0,0
for i in range(GRU.shape[0]):
  r1+=Metrica_GRU[i][0]
  r2+=Metrica_GRU[i][1]
  r3+=Metrica_GRU[i][2]

metrica_prueba=np.array([r1/(GRU.shape[0]), r2/(GRU.shape[0]), r3/(GRU.shape[0])])

In [ ]:
metrica_prueba

array([0.14321791, 0.02064463, 0.13725817])

#Transformers

##BERT

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 33.9 MB/s 
     |████████████████████████████████| 7.6 MB 62.3 MB/s 
     |████████████████████████████████| 182 kB 60.7 MB/s 


In [ ]:
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
from transformers import AutoModelForSeq2SeqLM
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForSeq2SeqLM.from_pretrained("EP9/bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum")
model.to(device)
tokenizer = BertTokenizerFast.from_pretrained("EP9/bert2bert_shared-spanish-finetuned-summarization-finetuned-xsum")
def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Downloading:   0%|          | 0.00/606 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

import sqlite3
dbfile = '/content/drive/MyDrive/NoticiasReforma2009.db'
import pandas as pd
import sqlite3
import sqlalchemy 

try:
    con = sqlite3.connect(dbfile)    
except Exception as e:
    print(e)

cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Table Name : {cursor.fetchall()}")


df = pd.read_sql_query('SELECT * FROM Noticias2009', con)
con.close()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Table Name : [('Reforma2009',), ('Noticias2009',)]


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

stopSpanish=set(stopwords.words('spanish'))
noticias=[]
for i in range (61035):
  text_tokens = word_tokenize(df['articulo'][i])
  tokens_without_sw = [word for word in text_tokens if not word in stopSpanish]
  tokens_without_sw = [word.lower() for word in tokens_without_sw if word.isalpha()]
  noticias.append(' '.join(tokens_without_sw))

stopSpanish=set(stopwords.words('spanish'))
resumen=[]
for i in range (len(df['resumen'])):
  text_tokens = word_tokenize(df['resumen'][i])
  tokens_without_sw = [word for word in text_tokens if not word in stopSpanish]
  tokens_without_sw = [word.lower() for word in tokens_without_sw if word.isalpha()]
  resumen.append(' '.join(tokens_without_sw))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
data=pd.DataFrame({'Noticia': noticias, 'Resumen Original': df['resumen'][:], 'Resumen Procesado': resumen})
data.to_csv('/content/drive/MyDrive/NoticiasProcesadas.csv')

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=2022)

In [ ]:
test=test.reset_index().drop(columns='index')
test=test.drop(columns='Resumen Original')

In [ ]:
test=test[test['Resumen Procesado'].notna()]

In [ ]:
test_rdd=sc.parallelize(test['Noticia'])

In [ ]:
test_rdd.collect()[0]

'acusan esposo crawford acosoel esposo supermodelo cindy crawford rande gerber acusado acoso sexual dos mujeres trabajaban dos meseras quejaron haber sido despedidas restaurante pertenece compañía gerber luego negarse entregársele acuerdo tmz demanda levantada marzo corte superior san diego alega gerber intentó tres ocasiones besar documento agrega hombre puso manos piernas mujeres salones san diego hard rock hotel los argumentos previamente investigados fundamento la demanda mérito manifestó vocero gerber cindy casada hace años dos hijos'

In [ ]:
generate_summary(test_rdd.collect()[0])

'el esposo supermodelo cindy crawford rande gerber acusado acoso sexual dos mujeres trabajaban dos meseras'

In [ ]:
resumen_generado=test_rdd.map(generate_summary).collect()

In [ ]:
test['Resumen New']=resumen_generado

In [ ]:
test.head()

,Noticia,Resumen Procesado,Resumen New
0,acusan esposo crawford acosoel esposo supermod...,el esposo supermodelo cindy crawford rande ger...,el esposo supermodelo cindy crawford rande ger...
1,padece cáncer esposo marcia crossel esposo mar...,el corredor bolsa tom mahoney esposo marcia cr...,el esposo marcia cross corredor bolsa tom maho...
2,descalifica muñoz ledo voto nulodurante gira a...,porfirio muñoz ledo candidato diputado criticó...,el candidato diputado plurinominal prd porfiri...
3,trasladan jackson féretro doradoel féretro bañ...,el féretro bañado oro contiene restos michael ...,el féretro bañado oro contiene restos michael ...
4,distingue gdf alcalde nueva orleansel alcalde ...,busca gobierno capitalino inversiones intercam...,el alcalde nueva orleans ray nagin declaró hué...


In [ ]:
test['Noticia'][50]

'preparación parejoel licenciado administración empresas uia ex presidente asociación egresados carrera jorge marroquín desempeña actualmente director desarrollo negocios empresa mancera ernst young méxico atribuye éxito tras años trayectoria haber combinado académico laboral joven en carreras administrativas común chicos estudien trabajen recomendable tener empleo egresar tan sólo años mientras cursaba tercer semestre carrera marroquín comenzó trabajar banco tan sólo seis meses después convirtió gerente tuve curso preparación puesto clientes sorprendían joven la universidad parte importante vida profesional en aprendí convivir personas diferentes carreras duda relaciones públicas base desarrollo sustento financiero dan estudios trato clientes aprende tiempo comenta pensar especialización maestría doctorado mejor chicos trabajen en experiencia puedo decir dentro mundo laboral estudiando cosas necesitaba gustaban hizo crecer dentro diferentes áreas dijo'

In [ ]:
test['Resumen Procesado'][50]

'jorge marroquín director desarrollo negocios mancera ernst young recomienda combinar estudios trabajo'

In [ ]:
test['Resumen New'][50]

'jorge marroquín director desarrollo negocios empresa mancera ernst young méxico atribuye éxito trayectoria'

In [ ]:
test.to_csv('/content/drive/MyDrive/ResultadoResumenesFINAL.csv')

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ResultadoResumenesFINAL.csv')
test=test.drop(columns='Unnamed: 0')

In [ ]:
test=test[test['Resumen Procesado']!='']
test=test.reset_index().drop(columns='index')

In [ ]:
pares=[]
for i in range(test.shape[0]):
  pares.append([test['Resumen New'][i], test['Resumen Procesado'][i]])

In [ ]:
BERT_rdd=sc.parallelize(pares)

In [ ]:
Metrica_BERT=BERT_rdd.map(Metrica).collect()

In [ ]:
r1,r2,r3=0,0,0
for i in range(test.shape[0]):
  r1+=Metrica_BERT[i][0]
  r2+=Metrica_BERT[i][1]
  r3+=Metrica_BERT[i][2]

metrica_prueba=np.array([r1/(test.shape[0]), r2/(test.shape[0]), r3/(test.shape[0])])

In [ ]:
metrica_prueba

array([0.49989792, 0.34141785, 0.46952882])

## MT5 SMALL

In [ ]:
import torch
from transformers import BertTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("EP9/mt5-small-tuto-mt5-small-2")
model.to(device)
tokenizer = BertTokenizerFast.from_pretrained("EP9/mt5-small-tuto-mt5-small-2")
def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

Downloading:   0%|          | 0.00/773 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test=pd.read_csv('/content/drive/MyDrive/ResultadoResumenesFINAL.csv')
test=test.drop(columns='Unnamed: 0')

In [ ]:
test.head()

,Noticia,Resumen Procesado,Resumen New
0,acusan esposo crawford acosoel esposo supermod...,el esposo supermodelo cindy crawford rande ger...,el esposo supermodelo cindy crawford rande ger...
1,padece cáncer esposo marcia crossel esposo mar...,el corredor bolsa tom mahoney esposo marcia cr...,el esposo marcia cross corredor bolsa tom maho...
2,descalifica muñoz ledo voto nulodurante gira a...,porfirio muñoz ledo candidato diputado criticó...,el candidato diputado plurinominal prd porfiri...
3,trasladan jackson féretro doradoel féretro bañ...,el féretro bañado oro contiene restos michael ...,el féretro bañado oro contiene restos michael ...
4,distingue gdf alcalde nueva orleansel alcalde ...,busca gobierno capitalino inversiones intercam...,el alcalde nueva orleans ray nagin declaró hué...


In [ ]:
test=test[test['Resumen Procesado']!='']
test=test[test['Resumen Procesado']!='.']
test=test.reset_index().drop(columns='index')

In [ ]:
test.head()

,Noticia,Resumen Procesado,Resumen New
0,acusan esposo crawford acosoel esposo supermod...,el esposo supermodelo cindy crawford rande ger...,el esposo supermodelo cindy crawford rande ger...
1,padece cáncer esposo marcia crossel esposo mar...,el corredor bolsa tom mahoney esposo marcia cr...,el esposo marcia cross corredor bolsa tom maho...
2,descalifica muñoz ledo voto nulodurante gira a...,porfirio muñoz ledo candidato diputado criticó...,el candidato diputado plurinominal prd porfiri...
3,trasladan jackson féretro doradoel féretro bañ...,el féretro bañado oro contiene restos michael ...,el féretro bañado oro contiene restos michael ...
4,distingue gdf alcalde nueva orleansel alcalde ...,busca gobierno capitalino inversiones intercam...,el alcalde nueva orleans ray nagin declaró hué...


In [ ]:
test_rdd=sc.parallelize(test['Noticia'])

In [ ]:
test_rdd.collect()[0]

'acusan esposo crawford acosoel esposo supermodelo cindy crawford rande gerber acusado acoso sexual dos mujeres trabajaban dos meseras quejaron haber sido despedidas restaurante pertenece compañía gerber luego negarse entregársele acuerdo tmz demanda levantada marzo corte superior san diego alega gerber intentó tres ocasiones besar documento agrega hombre puso manos piernas mujeres salones san diego hard rock hotel los argumentos previamente investigados fundamento la demanda mérito manifestó vocero gerber cindy casada hace años dos hijos'

In [ ]:
generate_summary(test_rdd.collect()[0])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'el esposo supermodelo cindy crawford acusado acoso sexual dos '

In [ ]:
resumen_generado1=test_rdd.map(generate_summary).collect()

In [ ]:
resumenes_MT5=[]
for i in range(test.shape[0]):
  print(i)
  aux=generate_summary(test['Noticia'][i])
  resumenes_MT5.append(aux)
  #print(aux)

In [ ]:
test['Resumen MT5']=resumenes_MT5

In [ ]:
test.head()

,Noticia,Resumen Procesado,Resumen New,Resumen MT5
0,acusan esposo crawford acosoel esposo supermod...,el esposo supermodelo cindy crawford rande ger...,el esposo supermodelo cindy crawford rande ger...,el esposo supermodelo cindy crawford acusado a...
1,padece cáncer esposo marcia crossel esposo mar...,el corredor bolsa tom mahoney esposo marcia cr...,el esposo marcia cross corredor bolsa tom maho...,el esposo marcia cross corredor bolsa tom maho...
2,descalifica muñoz ledo voto nulodurante gira a...,porfirio muñoz ledo candidato diputado criticó...,el candidato diputado plurinominal prd porfiri...,el líder prd convergencia trabajo salvemos méx...
3,trasladan jackson féretro doradoel féretro bañ...,el féretro bañado oro contiene restos michael ...,el féretro bañado oro contiene restos michael ...,el féretro bañado oro contiene restos michael ...
4,distingue gdf alcalde nueva orleansel alcalde ...,busca gobierno capitalino inversiones intercam...,el alcalde nueva orleans ray nagin declaró hué...,el alcalde nueva orleans ray nagin declarado h...


In [ ]:
pares=[]
for i in range(test.shape[0]):
  pares.append([test['Resumen MT5'][i], test['Resumen Procesado'][i]])

In [ ]:
MT5_rdd=sc.parallelize(pares)
Metrica_MT5=MT5_rdd.map(Metrica).collect()

In [ ]:
r1,r2,r3=0,0,0
for i in range(test.shape[0]):
  r1+=Metrica_MT5[i][0]
  r2+=Metrica_MT5[i][1]
  r3+=Metrica_MT5[i][2]

metrica_prueba=np.array([r1/(test.shape[0]), r2/(test.shape[0]), r3/(test.shape[0])])

In [ ]:
metrica_prueba

array([0.41663301, 0.28200058, 0.39611111])